# Kafka Demo

### Connect to Kafka Broker Server 
```
ssh -L <local_port>:localhost:<remote_port> <user>@<remote_server> -NTf
```
Find how to connect to Kafka server on Canvas lab 2 assignment page.

### To kill connection
```
lsof -ti:<local_port> | xargs kill -9
```

### Setup
I recommend setting up a python environment for this lab (and all other ones).
```
python -m venv <environment_name>
source <environment_name>/bin/activate
```
Then you can install the requirements.
```
python -m pip install kafka-python
```

In [ ]:
import os
from datetime import datetime
from json import dumps, loads
from time import sleep
from random import randint
from kafka import KafkaConsumer, KafkaProducer
from typing import Dict, Any

# TODO: Fill in a unique identifier so your topic doesn't collide with others'
andrew_id = ... # you can also use any unique identifier for yourself if you prefer :)
topic = f"lab02-{andrew_id}"
print(f"Topic: {topic}")

### Producer Mode -> Writes Data to Broker

In [ ]:
# I have provided the following schema for messages. You may change the city data if you wish but it is optional.
def make_city_data(city: str, temperature_f: str) -> Dict[str, Any]:
    return {
        "city": city,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "temperature_f": temperature_f, # temperature in fahrenheit
    }

In [ ]:
# Create a producer to write data to kafka
# Ref: https://kafka-python.readthedocs.io/en/master/apidoc/KafkaProducer.html

# [TODO]: Fill in the address of your Kafka bootstrap server
# [TODO]: Kafka expects messages as bytes. Explore the documentation and decide how to serialize Python dict objects into bytes.
# Hint: You may want to convert your Python dict → JSON string → UTF-8 bytes.

producer = KafkaProducer(bootstrap_servers=[...],
                        value_serializer=...)

In [ ]:
# [TODO]: Add a few more examples of city data below
cities = [make_city_data("Pittsburgh" , 64), make_city_data(... , ...), make_city_data(... , ...)]

print("Writing to Kafka Broker")
for i in range(10):
    data = cities[randint(0,len(cities)-1)] # random selection
    producer.send(topic=topic, value=data)
    sleep(1)

producer.flush()
print(f"Data written to topic: {topic}")

### Consumer Mode -> Reads Data from Broker

In [ ]:
# Create a consumer to read data from kafka
# Ref: https://kafka-python.readthedocs.io/en/master/apidoc/KafkaConsumer.html

# [TODO]: Complete the missing ... parameters/arguments using the Kafka documentation
consumer = KafkaConsumer(
    ...,
    bootstrap_servers=[...],
    auto_offset_reset=..., #Experiment with different values (earliest, latest, none)
    # Commit that an offset has been read
    enable_auto_commit=True,
    # How often to tell Kafka, an offset has been read
    auto_commit_interval_ms=1000
)

print('Reading Kafka Broker')
for message in consumer:
    message = message.value.decode()
    # Default message.value type is bytes!
    print(loads(message))
    os.system(f"echo {message} >> kafka_log.csv")

# Use kcat!
It's a CLI (Command Line Interface). Previously known as kafkacat


Ref: https://docs.confluent.io/platform/current/app-development/kafkacat-usage.html

In [ ]:
# [TODO]: Use kcat to consume the first 5 messages from your topic. Use the -f flag to include the offset in the output.
# Paste the output below and explain what the offset refers to.